# API development for download of Sentinel-2 and Landsat-8 data
## User defined mosaicking on harmonised products

Two different multi-spectral optical satellite constellations deliver open data source for earth observation.
The Sentinel-2 mission contains two identical satellites with suffix A and B, with opposite direction of orbits flying in a 
repeat cycle of 10 days each, 5 days for both orbits, respectively. Launch: 2015 (A), 2017 (B)[ESA].
<br>
As the second optical satellite Landsat 8 Operational Land Imager (OLI),...
with a repeat cycle of 16 days.  Launch: 2013, come with [res. etc.]... 2013
<br>


The scope of this extension to the python-based nasa_hls[email/github] aims to add the following functionality:
1. 


First, the module nasa_hls is loaded.

In [ ]:
%autoreload 2
%matplotlib inline

import nasa_hls
import sys

# and for later processing in the notebook

For testing purpose, try downloading the kml file:

In [ ]:
nasa_hls.download_kml()

make tiles from the user given:
1. spatial geometry
2. date or time span
3. product type (S30 or L30, or both)

The input features should come in the WGS84 coordinate system.
Dates have to be passes as yyyy-mm-dd strings

In [53]:
ds = nasa_hls.make_tiles_dataset(shape="/home/aleko-kon/Dokumente/nasa_hls/data/amazon.shp",
                                products=["S30"],
                                date="01-11-2018")



valid shape, process continues

single date: 01-11-2018
 
UTM tiles already successfully downloaded to:
 /home/aleko-kon/.nasa_hls/.auxdata/utm.kml 




  0%|          | 0/5 [00:00<?, ?it/s]


getting available datasets . . .



100%|██████████| 5/5 [00:08<00:00,  1.77s/it]


KeyError: datetime.date(2018, 1, 11)

In [47]:
import geopandas as gp
a = gp.read_file("/home/aleko-kon/Dokumente/nasa_hls/data/amazon.shp")

In [48]:
ds

NameError: name 'ds' is not defined